# Stellar interior model

Importamos las librerías necesarias para el código

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Secondary Functions

In [3]:
def energia(T,P,X,Y):
    """ 
    Calculate the energy production based on the local conditions.
    
    The function decides what is the predominant reaction (pp or CN).
    
    Parameters
    ---------- 
    T : Temperature. Input float.
    P : Pressure. Input float.
    X : Mass fraction of hydrogen.
    Y : Mass fraction of helium.
    
    Returns
    -------
    reac : It is a string. It says which type of reaction has been chosen.
        If reac is '--' it means there is no energy production.
    nu,e : Parameters to calculate the energy production in our conditions.
    """
    
    # Cálculo de la densidad
    Z=1-X-Y
    mu = 1/(2*X+0.75*Y+0.5*Z)
    H = 1/6.02214076e+23    # [g]
    boltz = 1.380649e-23    # [J/K]
    rho = mu*H/boltz*(P*10e8)/(T*10e7)
    
    # Rangos de temperatura
    # Cadena pp
    if T < 0.4:
        nu_pp = 0
        e_pp = 0
    elif T < 0.6:
        nu_pp = 6
        e_pp = 10**-6.84
    elif T < 0.95:
        nu_pp = 5
        e_pp = 10**-6.04
    elif T < 1.2:
        nu_pp = 4.5
        e_pp = 10**-5.56
    elif T < 1.65:
        nu_pp = 4
        e_pp = 10**-5.02
    elif T < 2.4:
        nu_pp = 3.5
        e_pp = 10**-4.40
    else:
        nu_pp = 0
        e_pp = 0
    
    # Ciclo CN
    if T < 1.2:
        nu_cn = 0
        e_cn = 0
    elif T < 1.6:
        nu_cn = 20
        e_cn = 10**-22.2
    elif T < 2.25:
        nu_cn = 18
        e_cn = 10**-19.8
    elif T < 2.75:
        nu_cn = 16
        e_cn = 10**-17.1
    elif T < 3.6:
        nu_cn = 15
        e_cn = 10**-15.6
    elif T < 5.0:
        nu_cn = 13
        e_cn = 10**-12.5
    else:
        nu_cn = 0
        e_cn= 0
        
    Epp = e_pp*(X**2)*rho*((T*10)**nu_pp)
    Ecn = e_cn*X*Z/3*rho*((T*10)**nu_cn)
    cual = (Ecn < Epp)      # Qué reacción domina?
    if cual == True:
        reac = 'pp'
        nu = nu_pp
        e = e_pp
    else:
        reac = 'CN'
        nu = nu_cn
        e = e_cn
    if e_pp==0:        # Se produce energía?
        reac = '--'
    return (reac,nu,e)


def FaseRad(X,Y,M_tot,R_tot,L_tot):
    """
    Calculate the boundary of the radiative part of the star.
    
    It makes an interpolation to find the expected values of radius, mass, pressure, luminosity and temperature at the point where the convective part starts.
    
    Parameters
    ----------
    X : mass fraction of hydrogen.
    Y : mass fraction of helium.
    M_tot : mass of the star.
    R_tot : radius of the star.
    L_tot : luminosity of the star.
    
    Returns
    -------
    h : step in radius.
    front : boundary layer.
    r_front : radius at the boundary.
    M_front : mass at the boundary.
    P_front : pressure at the boundary.
    L_front : luminosity at the boundary.
    T_front : temperature at the boundary.
    
    First values are usefull for FaseConv function.
    """
    # Inicialización de las variables
    R_in = 0.90*R_tot  # evitamos problemas de convergencia
    Z = 1-X-Y
    mu = 1/(2*X+0.75*Y+0.5*Z)   # Peso mulecular medio cte.
    n_capa = 101   # Número de capas en las que se va a dividir el radio
    
    #Inicializamos los vectores para guardar las variables
    capa = np.arange(0,n_capa,1)
    r = np.linspace(R_in,0,n_capa)
    M = np.zeros(n_capa)
    P = np.zeros(n_capa)
    L = np.zeros(n_capa)
    T = np.zeros(n_capa)
    N = np.zeros(n_capa)
    # Inicializamos las f_i para integración
    fm = np.zeros(n_capa)
    fp = np.zeros(n_capa)
    fl = np.zeros(n_capa)
    ft = np.zeros(n_capa)
    
    h = r[0]-r[1]   # Paso de integración
    
    # Capas superficiales (L=cte y M=cte)
    for i in range(3):
        M[i] = M_tot
        L[i] = L_tot
        T[i] = (1.9022*mu*M_tot)*(1/r[i]-1/R_tot)      # Se usa R_tot
        P[i] = (10.645*np.sqrt(M_tot/(mu*Z*(1+X)*L_tot)))*T[i]**4.25    
        # f para método diferencias (fm=fl=0 -> ctes). Caso radiativo
        ft[i] = -(0.01679*Z*(1+X)*mu**2)*(P[i]**2)*L[i]/((T[i]**8.5)*r[i]**2)
        fp[i] = -(8.084*mu)*P[i]*M_tot/(T[i]*r[i]**2)
        
    # Integración desde la superficie
    i = 2   # Las tres primeras capas ya están calculadas.
    N[i] = T[i]/P[i]*fp[i]/ft[i]  # Cálculo inicial de n+1
    
    if N[i]<=2.5:   # Comprobamos que las primeras capas sean radiativas
        raise NotImplementedError('La parte superficial de la estrella no es radiativa!')
    
    while N[i]>2.5:   # Capas radiativas hasta la frontera convectiva
        # Cálculo de las Delta_P[i] y Delta_T[i]
        DP1 = -h*fp[i]+h*fp[i-1]
        DP2 = -h*fp[i]+2*h*fp[i-1]-h*fp[i-2]
        DT1 = -h*ft[i]+h*ft[i-1]    # h es negativo
        # Cálculo P_est y T_est
        P_est = P[i]-h*fp[i]+(DP1/2)+(5/12*DP2)
        T_est = T[i]-h*ft[i]+DT1/2
        P_old = P[i]
        T_old = T[i]
        
        while abs((T_est-T_old)/T_est) > 0.0001:
            while abs((P_est-P_old)/P_est) > 0.0001:
                # Masa calculada
                fm[i+1] = 0.01523*mu*P_est*(r[i+1]**2)/T_est
                DM1 = -h*fm[i+1]+h*fm[i]    # Delta_M[i+1]
                M[i+1] = M[i]-h*fm[i+1]-DM1/2        
                #Presión calculada a partir de la anterior masa
                fp[i+1] = -(8.084*mu)*P_est/T_est*M[i+1]/(r[i+1]**2)
                DP1 = -h*fp[i+1]+h*fp[i]    # Delta_P[i+1]
                P[i+1] = P[i]-h*fp[i+1]-DP1/2  
                P_old = P_est
                P_est = P[i+1]
            
            # Producción energía
            t_reac,nu,e = energia(T_est,P[i+1],X,Y)    # Factores producción energía
            if t_reac == 'pp':
                fl[i+1] = 0.01845*e*(X**2)*(10**nu)*(mu**2)*(P[i+1]**2)*(T_est**(nu-2))*r[i+1]**2        
            elif t_reac =='CN':
                fl[i+1] = 0.01845*e*X*Z/3*(10**nu)*(mu**2)*(P[i+1]**2)*(T_est**(nu-2))*r[i+1]**2
            else:
                fl[i+1] = 0            
            # Luminosidad calculada
            DL1 = -h*fl[i+1]+h*fl[i]
            DL2 = -h*fl[i+1]+2*h*fl[i]-h*fl[i-1]
            L[i+1] = L[i]-h*fl[i+1]-(DL1/2)-(DL2/12)        
            # Temperatura calculada a partir de todo lo demás
            ft[i+1] = -0.01679*Z*(1+X)*mu**2*(P[i+1]**2)*(L[i+1])/((T_est**8.5)*r[i+1]**2)
            DT1 = -h*ft[i+1]+h*ft[i]
            T[i+1] = T[i]-h*ft[i+1]-DT1/2        
            T_old = T_est
            T_est = T[i+1]
            
        i+=1    # Contador
        if i > (n_capa-1):      # Para poner un límite al bucle while
            break

        N[i] = T[i]*fp[i]/(P[i]*ft[i])
        if N[i] <= 2.5:
            front = i    # Marcador del frontera radiativa
    # La última capa no es radiativa
    
    # INTERPOLACIÓN. Parte radiativa. Acortamos vectores a los dos últimos valores
    r = np.flip(r[frontera-1:frontera+1])
    M = np.flip(M[frontera-1:frontera+1])
    P = np.flip(P[frontera-1:frontera+1])
    L = np.flip(L[frontera-1:frontera+1])
    T = np.flip(T[frontera-1:frontera+1])
    N = np.flip(N[frontera-1:frontera+1])
    
    # Interpolamos valores de los parámetros para la capa n+1=2.5
    r_front = np.interp(2.5,N,r)
    M_front = np.interp(2.5,N,M)
    P_front = np.interp(2.5,N,P)
    L_front = np.interp(2.5,N,L)
    T_fornt = np.interp(2.5,N,T)
    
    return (h,front,r_front,M_front,P_front,L_front,T_front)


def FaseConv(X,Y,h,front,Tc,r_front,P_rad=0,T_rad=1):
    """
    Calculate the star parameters for the boundary layer of convection. It is based on an initial temperature which has to be given.
    
    Parameters
    ----------
    X : mass fraction of hydrogen.
    Y : mass fraction of helium.
    h : step in raidus. It has been computed in FaseRad function.
    front : boundary layer between radiation and convection. Also computed.
    Tc : central temperature of our star.
    r_front : expected radius on the boundary. Calculated in FaseRad.
    P_rad,T_rad : estimated pressure and temperature from radiative part on the boundary.
        It is used for calculate the constant K for the polytropic model of pressure. The default value is K=0.
        
    
    Returns
    -------
    M_front : mass at the boundary.
    P_front : preassure at the boundary.
    L_front : luminosity at the boundary.
    T_front : temperature at the boundary. 
    
    """
    
    # Inicialización de variables
    Z = 1-X-Y
    mu = 1/(2*X+0.75*Y+0.5*Z)
    K = P_rad/(T_rad**2.5)   # Polítropo
    
    # Inicialización de los vectores
    r = np.arange(0,r_front+1.1*h,h)   # Necesitamos un punto más allá de la frontera para interpolar
    M = np.zeros(len(r))
    P = np.zeros(len(r))
    L = np.zeros(len(r))
    T = np.zeros(len(r))
    # Inicializamos las f_i para método Predictor-Corrector
    fm = np.zeros(len(r))
    fl = np.zeros(len(r))
    ft = np.zeros(len(r))
    
    # Capa central (Mc=Lc=0). Calculamos las tres capas centrales
    for i in range(3):      # Tres capas centrales
        # Cálculo de los parámetros
        M[i] = 0.005077*mu*K*(Tc**1.5)*r[i]**3
        T[i] = Tc-0.008207*(mu**2)*K*(Tc**1.5)*r[i]**2
        P[i] = K*(T[i]**2.5)
        # Produción energía
        t_reac,nu,e = energia(Tc,P[0],X,Y)
        if t_reac == 'pp':
            L[i] = 0.00615*e*X**2*(10**nu)*mu**2*K**2*(Tc**(3+nu))*r[i]**3
        elif t_reac == 'CN':
            L[i] = 0.00615*e*X*Z/3*(10**nu)*mu**2*K**2*(Tc**(3+nu))*r[i]**3
        else:
            if i == 0:
                L[0] = 0
            else:
                L[i] = L[i-1]            
        # f para método diferencias
        fm[i] = 0.01523*mu*K*(T[i]**1.5)*(r[i]**2)   
        if r[i] != 0:    
            ft[i] = 3.234*mu*M[i]/(r[i]**2)     # en r=0 no se puede evaluar f[t]
        if t_reac == 'pp':
            fl[i] = 0.01845*e*X*X*(10**nu)*mu**2*K**2*(T[i]**(nu+3))*r[i]**2
        elif t_reac =='CN':
            fl[i] = 0.01845*e*X*Z/3*(10**nu)*mu**2*K**2*(T[i]**(nu+3))*r[i]**2
        else:
            fl[i] = 0
        
    # Integración desde el centro
    for i in range(2,len(r)-1):   # Las tres primeras capas ya están calculadas
        # Estimación de temperatura.
        DT1 = h*ft[i]-h*ft[i-1]
        T_est = T[i]+h*ft[i]+DT1/2
        T_old = T[i]
    
        while abs((T_est-T_old)/T_est) > 0.0001:
            # Presión estimada (polítropo) y masa calculada
            P_est = K*(T_est**2.5)
            fm[i+1] = 0.01523*mu*P_est*r[i+1]**2/T_est
            DM1 = h*fm[i+1]-h*fm[i]
            M[i+1] = M[i]+h*fm[i+1]-DM1/2
            # Cálculo de la nueva temperatura
            ft[i+1] = -3.234*mu*M[i+1]/r[i+1]**2
            DT1 = h*ft[i+1]-h*ft[i]
            T[i+1] = T[i]+h*ft[i+1]-DT1/2       
            T_old = T_est 
            T_est = T[i+1]
        
        P[i+1] = K*(T[i+1]**2.5)
        # Producción energía
        t_reac,nu,e = energia(T[i+1],P[i+1],X,Y)
        if t_reac == 'pp':
            fl[i+1] = 0.01845*e*X*X*(10**nu)*mu**2*(P[i+1]**2)*(T[i+1]**(nu-2))*r[i+1]**2
        elif t_reac == 'CN':
            fl[i+1] = 0.01845*e*X*Z/3*(10**nu)*mu**2*(P[i+1]**2)*(T[i+1]**(nu-2))*r[i+1]**2
        else:
            fl[i+1] = 0  
        # Cálculo de la luminosidad
        DL1 = h*fl[i+1]-h*fl[i]
        DL2 = h*fl[i+1]-2*h*fl[i]+h*fl[i-1]
        L[i+1] = L[i]+h*fl[i+1]-(DL1/2)-(DL2/12)
    # La última capa no es convectiva
    
    # INTERPOLACIÓN. Fase convectiva. Acortamos los vectores a los dos últimos valores
    r = r[len(T)-2:len(T)]
    M = M[len(T)-2:len(T)]
    P = P[len(T)-2:len(T)]
    L = L[len(T)-2:len(T)]
    T = T[len(T)-2:len(T)]
    
    # Interpolamos los valores de los parámetros para la capa n+1=2.5
    M_front = np.interp(r_front,r,M)
    P_front = np.interp(r_front,r,P)
    L_front = np.interp(r_front,r,L)
    T_front = np.interp(r_front,r,T)
    
    return(M_front,P_front,L_front,T_front)
